In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
# import ydf
import tensorflow_decision_forests as tfdf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"GPUs found: {gpus}")
else:
    print("No GPUs found.")

2025-10-11 13:24:04.385773: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760189044.562823      18 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760189044.614869      18 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


GPUs found: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [2]:
train_df=pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test_df=pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')

In [3]:
# preprocess?
def preprocess(df):
    # def cryoSleep_integer(x):
    #     return x.astype(int)
    # df=df.dropna()
    def normalize_name(x):
        x=str(x)
        return " ".join([v.strip(",()[].\"'") for v in x.split(" ")])
    def split_passenger_id(row):
        if pd.isna(row['PassengerId']):
            return pd.Series([None, None, None])
        arr=row['PassengerId'].split('_')
        if len(arr) < 2:
            arr += [None] * (3 - len(arr))
        
        return pd.Series(arr)
    def split_cabin(row):
        if pd.isna(row['Cabin']):
            return pd.Series([None, None, None])  # Handle None case
        arr = row['Cabin'].split('/')
        # Ensure arr has exactly 3 elements
        if len(arr) < 3:
            arr += [None] * (3 - len(arr))  # Pad with None
        return pd.Series(arr)
    def total_spent(row):
        return pd.Series(row['RoomService']+row['FoodCourt']+row['ShoppingMall']+row['Spa']+row['VRDeck'])
        

    # Apply the function and create new columns
    df.fillna({
    'RoomService': df['RoomService'].median(),
    'FoodCourt': df['FoodCourt'].median(),
    'ShoppingMall':df['ShoppingMall'].median(),
    'Spa':df['Spa'].median(),
    'VRDeck':df['VRDeck'].median(),
    # 'Deck':df['Deck'].median()
    }, inplace=True)
    df['Total_Spent']=df.apply(total_spent,axis=1)
    df["Name"] = df["Name"].apply(normalize_name)
    df[['Deck', 'Num', 'Side']] = df.apply(split_cabin, axis=1)
    df[['Group','Place_In_Group']]=df.apply(split_passenger_id,axis=1)
    df['CryoSleep'] = df['CryoSleep'].apply(lambda x: float(x))
    df=df.fillna({'CryoSleep':df['CryoSleep'].mean()})
    df['VIP'] = df['VIP'].apply(lambda x: float(x))

        
    # keep PassengerId and one-hot encode HomePlanet & Destination
    id_col = df['PassengerId']                     # actual column values
    dums = pd.get_dummies(df[['HomePlanet','Destination']], dummy_na=False)
    
    # drop the original encoded columns, then concat PassengerId + dummies + remaining columns
    remaining = df.drop(columns=['HomePlanet','Destination','PassengerId'])
    df = pd.concat([id_col.reset_index(drop=True), remaining.reset_index(drop=True), dums.reset_index(drop=True)], axis=1)
    
    # convert boolean columns to 0/1
    bool_cols = df.select_dtypes(include='bool').columns
    df[bool_cols] = df[bool_cols].astype(int)
    
    return df
    
train_df=preprocess(train_df)
test_df=preprocess(test_df)

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   PassengerId                8693 non-null   object 
 1   CryoSleep                  8693 non-null   float64
 2   Cabin                      8494 non-null   object 
 3   Age                        8514 non-null   float64
 4   VIP                        8490 non-null   float64
 5   RoomService                8693 non-null   float64
 6   FoodCourt                  8693 non-null   float64
 7   ShoppingMall               8693 non-null   float64
 8   Spa                        8693 non-null   float64
 9   VRDeck                     8693 non-null   float64
 10  Name                       8693 non-null   object 
 11  Transported                8693 non-null   int64  
 12  Total_Spent                8693 non-null   float64
 13  Deck                       8494 non-null   objec

In [5]:
train_df['Deck']

0       B
1       F
2       A
3       A
4       F
       ..
8688    A
8689    G
8690    G
8691    E
8692    E
Name: Deck, Length: 8693, dtype: object

In [6]:
train_df.head(5)

,PassengerId,CryoSleep,Cabin,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,...,Num,Side,Group,Place_In_Group,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,0001_01,0.0,B/0/P,39.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,P,0001,01,0,1,0,0,0,1
1,0002_01,0.0,F/0/S,24.0,0.0,109.0,9.0,25.0,549.0,44.0,...,0,S,0002,01,1,0,0,0,0,1
2,0003_01,0.0,A/0/S,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,...,0,S,0003,01,0,1,0,0,0,1
3,0003_02,0.0,A/0/S,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,...,0,S,0003,02,0,1,0,0,0,1
4,0004_01,0.0,F/1/S,16.0,0.0,303.0,70.0,151.0,565.0,2.0,...,1,S,0004,01,1,0,0,0,0,1


In [7]:

# train_df[bool_cols].fillna()

In [8]:
train_df

,PassengerId,CryoSleep,Cabin,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,...,Num,Side,Group,Place_In_Group,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,0001_01,0.0,B/0/P,39.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,P,0001,01,0,1,0,0,0,1
1,0002_01,0.0,F/0/S,24.0,0.0,109.0,9.0,25.0,549.0,44.0,...,0,S,0002,01,1,0,0,0,0,1
2,0003_01,0.0,A/0/S,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,...,0,S,0003,01,0,1,0,0,0,1
3,0003_02,0.0,A/0/S,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,...,0,S,0003,02,0,1,0,0,0,1
4,0004_01,0.0,F/1/S,16.0,0.0,303.0,70.0,151.0,565.0,2.0,...,1,S,0004,01,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,0.0,A/98/P,41.0,1.0,0.0,6819.0,0.0,1643.0,74.0,...,98,P,9276,01,0,1,0,1,0,0
8689,9278_01,1.0,G/1499/S,18.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1499,S,9278,01,1,0,0,0,1,0
8690,9279_01,0.0,G/1500/S,26.0,0.0,0.0,0.0,1872.0,1.0,0.0,...,1500,S,9279,01,1,0,0,0,0,1
8691,9280_01,0.0,E/608/S,32.0,0.0,0.0,1049.0,0.0,353.0,3235.0,...,608,S,9280,01,0,1,0,1,0,0


In [9]:

input_features=list(train_df.columns)
input_features.remove('Transported')
input_features.remove('PassengerId')
input_features.remove('Cabin')
# input_features=['CryoSleep']
input_features.remove('Name')
input_features.remove('Side')
input_features.remove('Deck')

In [10]:
train_df=train_df.fillna(0)

In [11]:
from sklearn.preprocessing import StandardScaler



# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(len(input_features),)),
    tf.keras.layers.Dense(128, activation='relu'),  # Increased complexity
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='SGD', 
              loss = tf.keras.losses.BinaryFocalCrossentropy(),
              metrics=['accuracy'])

# Prepare the training data
X_train = train_df[input_features].values
y_train = train_df['Transported'].astype(int).values
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
# Fit the model
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)

X_test = test_df[input_features].copy()
X_test = X_test.apply(pd.to_numeric, errors='coerce')
X_test = X_test.fillna(X_test.median())   # or other strategy
X_test = scaler.transform(X_test)        # use same scaler you fitted
predictions = model.predict(X_test)

I0000 00:00:1760189062.766437      18 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1760189062.767141      18 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Epoch 1/50


I0000 00:00:1760189064.984533      61 service.cc:148] XLA service 0x7fc124010340 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1760189064.985031      61 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1760189064.985049      61 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1760189065.171847      61 cuda_dnn.cc:529] Loaded cuDNN version 90300


 77/218 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5450 - loss: 0.1830

I0000 00:00:1760189065.936900      61 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


218/218 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.5927 - loss: 0.1718 - val_accuracy: 0.7579 - val_loss: 0.1376
Epoch 2/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7313 - loss: 0.1402 - val_accuracy: 0.7775 - val_loss: 0.1268
Epoch 3/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7539 - loss: 0.1293 - val_accuracy: 0.7809 - val_loss: 0.1225
Epoch 4/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7617 - loss: 0.1244 - val_accuracy: 0.7867 - val_loss: 0.1206
Epoch 5/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7777 - loss: 0.1186 - val_accuracy: 0.7930 - val_loss: 0.1183
Epoch 6/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7639 - loss: 0.1211 - val_accuracy: 0.7930 - val_loss: 0.1181
Epoch 7/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7714 - loss: 0.1178 - val_accuracy: 0.7913 - val_loss: 0.1160
Epoch 8/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7768 - loss: 0.1176 - val_accuracy: 0.7947 - val_

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


In [12]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         2,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,803 (50.02 KB)

 Trainable params: 12,801 (50.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [13]:
def prediction_to_kaggle_format(model, test_df, threshold=0.5):
    # Prepare features for predictions using the same input features as used in training
    # X_test = test_df[input_features].values  # Ensure input features are consistent
    proba_survive = model.predict(X_test, verbose=0)  # Predict probabilities
    
    # Check the shape of proba_survive
    if proba_survive.ndim > 1:
        proba_survive = proba_survive[:, 0]  # Flatten if necessary
    
    # Create the DataFrame for submission
    return pd.DataFrame({
        "PassengerId": test_df["PassengerId"].values,  # Ensure this is a 1D array
        "Transported": (proba_survive >= threshold).astype(str)  # Convert boolean to string
    })

def make_submission(kaggle_predictions):
    path = "/kaggle/working/submission.csv"
    kaggle_predictions.to_csv(path, index=False)
    print(f"Submission exported to {path}")

# Generate predictions and make the submission
kaggle_predictions = prediction_to_kaggle_format(model, test_df)  # Use preprocessed test_df
make_submission(kaggle_predictions)

# Optional: Check the first few lines of the submission
!head /kaggle/working/submission.csv

Submission exported to /kaggle/working/submission.csv
PassengerId,Transported
0013_01,False
0018_01,False
0019_01,True
0021_01,True
0023_01,False
0027_01,True
0029_01,True
0032_01,True
0032_02,True
